In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#記得掛載雲端硬碟 
from zipfile import ZipFile
path = "/content/drive/MyDrive/ttgroup/0_9000/0519_out_resize256x256_0_9000_json.zip"
f = ZipFile(path)
# f.extractall() 小括號是直接解壓縮在同一層
f.extractall()


In [ ]:
# img_fn_df 圖片名稱DataFrame
import glob
import json, codecs
import pandas as pd
import numpy as np

fn_dic = {"img_name":[]}
# fn_dic_2 = []
paths = sorted(glob.glob("0519_out_resize256x256_0_9000_json/*"))
for path in paths:
  fn_type = path.split("/")[-1]
  fn = fn_type.split(".")[0]
  fn_dic["img_name"].append(fn)
  # fn_dic_2.append(fn)
# print(fn_dic)
fn_dic_df = pd.DataFrame(fn_dic)
fn_dic_df
# print(fn_dic_2)


# file_path = "/content/1162.json"
# for i in range(len(fs))
# obj_text = codecs.open(paths[0], 'r', encoding='utf-8').read()
# b_new = json.loads(obj_text)
# a_new = np.array(b_new)
# a_new

In [ ]:
# ans_df  驗證資料DataFrame
import os
base = "/content/drive/MyDrive/ttgroup/0_9000/0519_wrong_total_sit_value_combinefoot_0_9000.csv"
sitting_csv = pd.read_csv(base, sep=",")
ans_df = pd.DataFrame(sitting_csv)
ans_df = ans_df.drop(["img_name"], axis=1)
ans_df
# ans = df[:1]
# ans = ans.drop(["img_name"], axis=1)
# ans
# print(type(sitting_csv)
# np.array(sitting_csv)
# ans_head_df = pd.DataFrame(ans_df["head"])


In [ ]:
# keypoints_36_combine_df 關節節點DataFrame
import json, codecs
import numpy as np
columns ={
     "coco_col":['Nose_x', 'Nose_y', 'Neck_x', 'Neck_y',
           'R-Sho_x', 'R-Sho_y', 'R-Elb_x', 'R-Elb_y',
           'R-Wr_x', 'R-Wr_y', 'L-Sho_x', 'L-Sho_y', 
           'L-Elb_x', 'L-Elb_y', 'L-Wr_x', 'L-Wr_y',
           'R-Hip_x', 'R-Hip_y', 'R-Knee_x', 'R-Knee_y',
           'R-Ank_x', 'R-Ank_y', 'L-Hip_x', 'L-Hip_y',
           'L-Knee_x', 'L-Knee_y', 'L-Ank_x', 'L-Ank_y',
           'R-Eye_x', 'R-Eye_y', 'L-Eye_x', 'L-Eye_y',
           'R-Ear_x', 'R-Ear_y', 'L-Ear_x', 'L-Ear_y']
    }
fn_list = fn_dic["img_name"]
# print(fn_list) 印0004, 0006...

# 運用globals()[] 全域變數：做成動態的變數名。
n_fn_list = []
for fn in fn_list:
  # print(fn)
  obj_text = codecs.open("/content/0519_out_resize256x256_0_9000_json/"+ fn + ".json", 'r', encoding='utf-8').read()
  b_new = json.loads(obj_text)
  a_new = np.array(b_new)
  a_new = a_new.reshape(1,36)
  # print(a_new)
  globals()["key_point_36"+ fn +"_df"] = pd.DataFrame(a_new, columns = columns["coco_col"])
  # print(globals()["key_point_36"+ fn +"_df"])
  n_fn_list.append(globals()["key_point_36"+ fn +"_df"])
# print(n_fn_list)

# pd.concat([key_point_36_0004_df, key_point_36_0006_df, key_point_36_0008_df])
key_point_36_combine_df = pd.concat(n_fn_list)
key_point_36_combine_df

# bond.reset_index(inplace=True)
key_point_36_combine_df = key_point_36_combine_df.reset_index()
key_point_36_combine_df

# df.drop(columns=['B', 'C'])
# df.drop(['B', 'C'], axis=1)
key_point_36_combine_df = key_point_36_combine_df.drop(columns=['index'])
key_point_36_combine_df


In [ ]:
# key_point_36_combine_df, class_name_df
pd.concat([key_point_36_combine_df, ans_df], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

x_train, x_test, y_train, y_test = train_test_split(np.array(key_point_36_combine_df),
                             np.array(ans_df),
                             test_size=0.1)
# x_train, x_test, y_train, y_test  

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
#　調整Dense 第一個參數為256
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 調整參數
DENSE = 32
layers = [
          # 36 * 128 + 128 bias (對activation做前置偏移調整)
          Dense(DENSE, activation = "relu", input_dim = 36),
          # 128 * 3 + 3 bias (對activation做前置偏移調整)
          Dense(3, activation = "sigmoid")
]
model = Sequential(layers)
model.summary()

In [ ]:
#　設定Loss，用model.compile
# 如果是一個神經元(要視為兩個機率來做): Binary(二元的) 用 BinaryCrossentropy
# 多個神經元: sigma(pi*log(1/qi)): CategoricalCrossEntropy
# 這邊有多個神經元．所以用CategoricalCrossEntropy
from tensorflow.keras.losses import BinaryCrossentropy
# model.compile(loss="mse",
# 這邊不用MES因為坡度變化太小，很難看到誤差，所以用CrossEntropy來設定目標 
model.compile(loss=BinaryCrossentropy(),
       metrics=["accuracy"],
       optimizer="adam")
# optimizer 優化器：動量優化，解決局部最小值的問題，像彈珠
#           動態步長,前面走大步一點，後面越走越小步，快到目的地了。(像找門牌)
# 兩個結合起來即"adam"

In [ ]:
# # 用to_categorical 做one-hot-encodin, 將訓練跟測試的答案分別放到to_categorical 做分類(y_train),to_categorical(y_test)  
# # 記得num_classes要帶入,這邊[0]~[9],共10種所以放入10, 做完之後用訓練跟測試的答案命名
# # num_classes=10，強迫給出10個機率，以免資料錯誤
# from tensorflow.keras.utils import to_categorical
# # y_train_cat = to_categorical(y_train, num_classes=10)
# # y_test_cat = to_categorical(y_test, num_classes=10)
# y_train_cat = to_categorical(y_train, 10)
# y_test_cat = to_categorical(y_test, 10)
# print(y_train[0]) #改過前
# print(y_train_cat[0]) #改過後　one-hot-encoding

In [ ]:
# # reshape 將訓練跟測試資料的題目改為一維,原本是(60000, 28, 28)→(-1, 784)
# # 1-1-1
# # x_train.reshape(60000, 784)
# # x_test.reshape(10000, 784)
# # 1-1-2
# # # 可以利用reshape隱藏參數[-1]：60000，784其中一個也可寫成-1（代表全部數量）,請機器自己算。 #不能兩個參數都調-1
# # x_train.reshape(-1, 784)
# # x_test.reshape(-1, 784)
 
# # 將散佈範圍調整至-1至1之間，tensorflow的input_range的淺規則是在(-1,1) or (0,1)　之間；
# # 在以前caffe函式庫分析圖片時不用做轉換，因為本來input_range就是(0, 255)
# x_train_shape = x_train.reshape(-1, 784) / 255.0
# x_test_shape = x_test.reshape(-1, 784) / 255.0
# x_train_shape

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callbacks = [
   EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train,
     y_train,
     batch_size=20,
     epochs=50,
     validation_split=0.1,
     verbose=2,
     callbacks=callbacks)

In [ ]:
# 用model.evaluate評估模型 
# evaluate完去調dense 128 -> 256/64/512去找到最佳, 最後256跟512差不多,那就用256, 用越少的神經元學習效率較高
model.evaluate(x_test, y_test)

Dense:128
batch_size:15
val_accuracy:0.56, 0.5, 0.43, 0.54, 0.47, 0.56, 0.69
val_loss:1.6, 1.55, 2.23, 1.41 2.17, 1.23, 1.43, 2.08

In [ ]:
ans_names = ans_df.columns.tolist()
# key_point_36_combine_columns_list
key_point_36_combine_columns_list = key_point_36_combine_df.columns

In [ ]:
# model.predict()
# input 
# x_test[:][0].shape
# x_test_r = x_test[:][0].reshape(1,36) 
# pre = model.predict(x_test_r)
# pre_list = pre[0].tolist()
# for i, ans_name in enumerate(ans_names):
#   print(ans_name + ":", round(pre_list[i]))
  
# x_test[:][0].shape
# pre_df = pd.DataFrame(pre)
# pre_df[0].apply(round)
# for i in range(4):
#   print(pre_df[i].apply(round))


In [ ]:
# input_test
fn = input("請輸入檔名:")
obj_text = codecs.open(fn, 'r', encoding='utf-8').read()
b_new = json.loads(obj_text)
a_new = np.array(b_new)
a_new = a_new.reshape(1,36)
input_key_point_36_df = pd.DataFrame(a_new, columns=key_point_36_combine_columns_list)
input_key_point_36_df

# input_key_point_36_df.shape

# 印出head, shoulder, foot 是 1 的機率
x_test_in = input_key_point_36_df
x_test_in
pre = model.predict(x_test_in)[0]

# 將機率清單轉成T&F
ans_names = ans_df.columns.tolist()
pre_list = pre.tolist()
print("預測數值", pre_list)

ans_list = []
ans_name_list = []
for i, ans_name in enumerate(ans_names):
  print(ans_name + ":", round(pre_list[i]))
  ans_list.append(round(pre_list[i]))
  ans_name_list.append(ans_name)
# print(ans_list)
ans_trans = []
for i, ans_val in enumerate(ans_list):
  if ans_val == 1:
    ans_trans.append(ans_name_list[i] + "_wrong")
  else:
    ans_trans.append(ans_name_list[i] + "_correct")
print(ans_trans)